In [4]:
'''
Description: 
Version: 1.0
Autor: Yaxiong Shao
Date: 2022-06-22 18:10:29
LastEditors: Yaxiong Shao
LastEditTime: 2022-06-24 14:52:42
'''
import pandas as pd
import geopandas as gpd
import glob

## Join Geoid Back to results

In [5]:
index_df=pd.read_csv("../17031-Block-Car-OD-index_166987_V2.csv")
index_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../17031-Block-Car-OD-index_166987_V2.csv'

In [ ]:
acc_df=pd.read_csv("access30_min_f.csv")
acc_df.fillna(0,inplace=True)
acc_df["acc30_1000"]=acc_df["acc30"]*1000
acc_df.sort_values(by="orig", axis=0, ascending=True, inplace=True)
acc_df.head()

,orig,acc30,acc30_1000
23961,1,0.004376,4.375594
55912,2,0.004319,4.318673
57764,3,0.004250,4.250088
52549,4,0.004267,4.266677
12906,5,0.004246,4.246415


In [ ]:
#join
df=index_df.merge(acc_df,left_on="id",right_on="orig")
df.drop(["id"],axis=1,inplace=True)
df.head()

,geoid,orig,acc30,acc30_1000
0,170310101001000,1,0.004376,4.375594
1,170310101001001,2,0.004319,4.318673
2,170310101001002,3,0.004250,4.250088
3,170310101001003,4,0.004267,4.266677
4,170310101002000,5,0.004246,4.246415


## Join Shp to Results

In [ ]:
cook_gdf=gpd.read_file("Blocks.shp")
cook_gdf.drop(["TRACTCE10","HOUSING10","LAT","LON"],axis=1,inplace=True)
cook_gdf.rename(columns={"GEOID10":"geoid"},inplace=True)
cook_gdf["geoid"] = pd.to_numeric(cook_gdf["geoid"])
cook_gdf.head()

,geoid,POP10,geometry
0,170318046091014,88,"POLYGON ((-88.04667 42.03609, -88.04655 42.036..."
1,170318234004046,4,"POLYGON ((-87.70602 41.66378, -87.70557 41.663..."
2,170318226022022,69,"POLYGON ((-87.74494 41.69999, -87.74494 41.699..."
3,170318232002032,2,"POLYGON ((-87.73946 41.66173, -87.73964 41.661..."
4,170318233023013,0,"POLYGON ((-87.71278 41.65982, -87.71278 41.659..."


In [ ]:
cook_gdf.shape

(99032, 3)

In [ ]:
cook_gdf.dtypes

geoid          int64
POP10          int64
geometry    geometry
dtype: object

In [ ]:
res_gdf=cook_gdf.merge(df,on="geoid")
res_gdf.head()

,geoid,POP10,geometry,orig,acc30,acc30_1000
0,170318046091014,88,"POLYGON ((-88.04667 42.03609, -88.04655 42.036...",48951,0.002810,2.809848
1,170318234004046,4,"POLYGON ((-87.70602 41.66378, -87.70557 41.663...",75122,0.003283,3.283142
2,170318226022022,69,"POLYGON ((-87.74494 41.69999, -87.74494 41.699...",74010,0.003895,3.895476
3,170318232002032,2,"POLYGON ((-87.73946 41.66173, -87.73964 41.661...",74701,0.002504,2.503726
4,170318233023013,0,"POLYGON ((-87.71278 41.65982, -87.71278 41.659...",74811,0.003323,3.322634


In [ ]:
res_gdf.to_file("access30_min_f.geojson",driver="GeoJSON")
res_gdf.to_file("access30_min_f.shp")
res_gdf.to_file('access.gpkg', driver='GPKG', layer='access30_min_f')  

/Users/yaxiongshao/opt/miniconda3/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/yaxiongshao/opt/miniconda3/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/yaxiongshao/opt/miniconda3/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Batch Processing results

In [9]:
import os
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')

index_df=pd.read_csv("../1.Supply/data/17031-Block-Car-OD-index_166987_V2.csv")
index_df.head()

csv_list=glob.glob("2.RawResults/*.csv")
for csv in csv_list:
    print("processing:",csv)
    path, file_name=os.path.split(csv)
    file_name=file_name.replace(".csv","")
    acc_df=pd.read_csv(csv)
    acc_df.fillna(0,inplace=True)
    acc_field=list(acc_df.columns)
    #print(list(acc_df.columns))
    print(acc_field[1])

    acc_df[acc_field[1]+"_1000"]=acc_df[acc_field[1]]*1000
    


    acc_df.sort_values(by="orig", axis=0, ascending=True, inplace=True)

    df=index_df.merge(acc_df,left_on="id",right_on="orig")
    df.drop(["id"],axis=1,inplace=True)

    cook_gdf=gpd.read_file("../1.Supply/data/Census2010/CookCountyBlocks/Blocks.shp")
    cook_gdf.drop(["TRACTCE10","HOUSING10","LAT","LON"],axis=1,inplace=True)
    cook_gdf.rename(columns={"GEOID10":"geoid"},inplace=True)
    cook_gdf["geoid"] = pd.to_numeric(cook_gdf["geoid"])
    cook_gdf.head()

    res_gdf=cook_gdf.merge(df,on="geoid",how="left")
    res_gdf.fillna(0,inplace=True)

    #Normalize, after joined with shp file
    # Max = np.max(acc_df[acc_field[1]])
    # Min = np.min(acc_df[acc_field[1]])
    Max = res_gdf[acc_field[1]].max()
    Min = res_gdf[acc_field[1]].min()
    print(Max,Min)

    
    res_gdf[acc_field[1]+'_n'] = (res_gdf[acc_field[1]] - Min)/(Max - Min)
    res_gdf.head()

    res_gdf.to_file("3.Results_geojson/"+file_name+".geojson",driver="GeoJSON")
    res_gdf.to_file("4.Results_shp/"+file_name+".shp")
    res_gdf.to_file('access.gpkg', driver='GPKG', layer=file_name)
    res_gdf.drop(["geometry","orig"],axis=1,inplace=True)
    res_gdf.to_csv("5.Results_csv_without15mi/"+file_name+".csv",index=False)
    #break;

processing: 2.RawResults/access_2700_max_f.csv
acc2700
0.0055267078724659 5.896035155996315e-05
processing: 2.RawResults/access_900_1.csv
acc900
0.0079443003137612 0.0
processing: 2.RawResults/access_2700_1.csv
acc2700
0.0055403911129974 5.8959168336359855e-05
processing: 2.RawResults/access_1800_1.csv
acc1800
0.0064796149346826 0.0
processing: 2.RawResults/access_900_min_a.csv
acc900
0.0061433594300938 0.0
processing: 2.RawResults/access_1800_min_f.csv
acc1800
0.0064822144031215 0.0
processing: 2.RawResults/access_3600_min_f.csv
acc3600
0.0043267896528682 0.0
processing: 2.RawResults/access_3600_1.csv
acc3600
0.004323305663625 0.0017152272468776
processing: 2.RawResults/access_3600_min_a.csv
acc3600
0.0028846287984337 0.0
processing: 2.RawResults/access_1800_min_a.csv
acc1800
0.00457487955859 0.0
processing: 2.RawResults/access_900_min_f.csv
acc900
0.0079585129431082 0.0
processing: 2.RawResults/access_1800_max_f.csv
acc1800
0.0064704999935894 0.0
processing: 2.RawResults/access_3600_

## Visualize Results